In [ ]:
%load_ext autoreload
%autoreload 2/

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"    

import sys
sys.path.append("../..")
sys.path.append("..")

In [ ]:
# import tensorflow as tf
# tf.enable_eager_execution()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import distributions as dist
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

from torchvision.datasets import MNIST, FashionMNIST
from torchvision import transforms as tr
from torchvision.models import vgg11_bn
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

from pathlib import Path
import pickle
from torch import nn
from inpainting.datasets.mnist import train_val_datasets
from inpainting.visualizations.digits import digit_with_mask as vis_digit_mask
from inpainting.visualizations import visualizations_utils as vis
from inpainting.training import train_inpainter
from inpainting.utils import classifier_experiment, inpainted, predictions_for_entire_loader
from inpainting.datasets import mask_coding as mc
from inpainting.datasets.utils import RandomRectangleMaskConfig
from inpainting.custom_layers import Reshape
from inpainting.losses import nll_masked_batch_loss, nll_masked_batch_loss_components, r2_masked_batch_loss, nll_masked_batch_loss_same_size_masks

# from fancyimpute import NuclearNormMinimization, SoftImpute, BiScaler, MatrixFactorization, IterativeSVD, SoftImpute
# from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer
import tensorflow as tf

In [ ]:
import ACFlow
from ACFlow.utils.hparams import HParams
from ACFlow.models import get_model
from ACFlow.datasets import get_dataset
from ACFlow.datasets.dummy import Dataset

In [ ]:
acflow_params = HParams("../../ACFlow/exp/mnist/rnvp/params.json")
acflow_params.dict

In [ ]:

trainset = Dataset(acflow_params)
validset =  Dataset(acflow_params)
testset = Dataset(acflow_params)

model = get_model(acflow_params)
model.build(trainset, validset, testset)


In [ ]:
saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

weights_dir = os.path.join("../../ACFlow", acflow_params.exp_dir, 'weights')
# logging.info(f'Restoring parameters from {weights_dir}')
restore_from = tf.train.latest_checkpoint(weights_dir)
saver.restore(sess, restore_from)


In [ ]:
tf.saved_model.simple_save(
    sess,
    "acflow",
    {},
    outputs,
    legacy_init_op=None
)

In [ ]:
class ACFlowModule(tf.Module):
    
    def __init__(self, model):
        self.model = model
    
    @tf.function
    def inverse_zero(self, x, j):
        return self.model.inverse_zero(x, j, False)
        

In [ ]:
# testset.initialize(sess)

In [ ]:
ds_train, ds_val = ds_train, ds_val = train_val_datasets(
    Path.home() / "uj/.data/",
    mask_configs=[
        RandomRectangleMaskConfig(mc.UNKNOWN_LOSS, 14, 14, 0,0),
    ],
    ds_type=MNIST,
    resize_size=(28,28),
)


In [ ]:
X = []
J = []

for i, ((x, j), y) in enumerate(ds_val):
    X.append(x)
    J.append(j)
    
    if i ==63:
        break
    
X = torch.stack(X)
J = torch.stack(J)

X.shape, J.shape

In [ ]:
X_r, J_r = [torch.nn.functional.interpolate(t, (32, 32)).numpy() for t in [X, J]]

X_r.shape, J_r.shape

In [ ]:
X_t, J_t = [
        tf.constant(t.transpose(0, 2, 3, 1).astype("uint8")) # BCHW -> BHWC
    for t in [X_r*255, J_r]
]

print(X_t, J_t)

with tf.variable_scope('model', reuse=tf.AUTO_REUSE):
    inpainted_custom = model.inverse_zero(X_t, J_t, False)

inp_np, X_np, J_np = sess.run(
        [inpainted_custom, X_t, J_t]
)
    


In [ ]:
for i, (x, j, inp) in enumerate(zip(X_np, J_np, inp_np)):
    fig, ax = plt.subplots(1, 4)
    
    ax[0].imshow(x.squeeze())
    ax[1].imshow(j.squeeze())
    ax[2].imshow((x*j).squeeze())
    ax[3].imshow(inp.squeeze())
    plt.show()
    
    if i>=10:
        break

In [ ]:
testset.x, testset.m

In [ ]:
with tf.variable_scope('model', reuse=tf.AUTO_REUSE):
    inpainted = model.inverse_zero(testset.x, testset.m, False)

sam, gt, mask = sess.run(
        [inpainted, testset.x, testset.m])

In [ ]:
for i, (g, m, s) in enumerate(zip(gt, mask, sam)):
    fig, ax = plt.subplots(1, 4)
    
    ax[0].imshow(g.squeeze())
    ax[1].imshow(m.squeeze())
    ax[2].imshow((g*m).squeeze())
    ax[3].imshow(s.squeeze())
    plt.show()
    
    if i>= 10:
        break